# Create a table of global mean surface air temperature (GMST) for each CMIP6 model, ensemble member, scenario, and year



In [1]:
import numpy as np
import pandas as pd
import xarray as xr
import fsspec
from file_control import gmst_table_dir
from file_control import model_table_dir

In [2]:
outdir = gmst_table_dir

# Open the CMIP6 zarr data catalog 
df = pd.read_csv('https://storage.googleapis.com/cmip6/' + 
                 'cmip6-zarr-consolidated-stores.csv')

# get list of models to use
wr = pd.read_csv(model_table_dir + 'all_zarr_models.csv')
wr = wr.drop_duplicates()
outfile = outdir + 'CMIP6_GMST_table_all.csv'
scenarios = ['historical','ssp119','ssp126','ssp245','ssp370','ssp434','ssp460',
             'ssp534-over','ssp585'] 
        
mods = wr['model'].tolist()
    

In [3]:
# view the table of model/institution/ensemble member combinations to get data 
# for
wr

,model,institution,member
0,GFDL-CM4,NOAA-GFDL,r1i1p1f1
2,GFDL-ESM4,NOAA-GFDL,r1i1p1f1
3,GFDL-ESM4,NOAA-GFDL,r3i1p1f1
4,GFDL-ESM4,NOAA-GFDL,r2i1p1f1
9,IPSL-CM6A-LR,IPSL,r3i1p1f1
...,...,...,...
1695,NorESM2-LM,NCC,r7i1p1f2
1696,NorESM2-LM,NCC,r4i1p1f2
1697,NorESM2-LM,NCC,r3i1p1f2
1698,NorESM2-LM,NCC,r1i1p1f2


In [4]:
# how many unique models are we getting data for?
wr['model'].drop_duplicates().shape

(50,)

In [5]:
# Create a pandas dataframe to fill in with the GMST data
outdf = pd.DataFrame(columns=['model','member','scenario','year','GMST',
                              '1850-1900'])
outdf

,model,member,scenario,year,GMST,1850-1900


In [6]:
# function to create spatially weighted means
# adapted from 
# https://docs.xarray.dev/en/stable/examples/area_weighted_temperature.html
def spatial_mean(xx):
    weights = np.cos(np.deg2rad(xx.lat))
    weights.name = "weights"
    xx_weighted = xx.weighted(weights)
    weighted_mean = xx_weighted.mean(("lon", "lat"))
    return weighted_mean

In [ ]:

for m in range(len(mods)):
    perc_str=str(m/len(mods)*100) + '% done' 
    print(perc_str, end='\r')
    wrm = wr.iloc[m]

    # reset base
    base = np.nan
    
    for s in range(len(scenarios)):
        scen = scenarios[s]

        # find the model and scenario in the zarr data

        # see if data is available from zarr
        df_zarr = df.query("table_id == 'Amon' and variable_id == 'tas' and " + 
                           "experiment_id == '" + scen + "' and " + 
                           "institution_id == '" + wrm.institution + "' and " + 
                           "source_id == '" + wrm.model + "' and " + 
                           "member_id == '" + wrm.member + "'")
        
        # catch special case where MPI-ESM1-2-HR isn't found. Trying using MPI-M
        # institution instead
        if ((len(df_zarr)==0) and (wrm.model == 'MPI-ESM1-2-HR')):
            df_zarr = df.query("table_id == 'Amon' and " +
                               "variable_id == 'tas' and " +
                               "experiment_id == '" + scen + "' and " + 
                               "institution_id == 'MPI-M' and " + 
                               "source_id == '" + wrm.model + "' and " +
                               "member_id == '" + wrm.member + "'")

                                 
        if (len(df_zarr)==0): 
            1
            #print('not available')
        else:
            # first grab the historical period data
            zstore = df_zarr.zstore.values[0]
            # create a mutable-mapping-style interface to the store
            mapper = fsspec.get_mapper(zstore)
            # open it using xarray and zarr
            zz = xr.open_zarr(mapper, consolidated=True)
            # make sure time is sorted
            zz = zz.sortby('time')
            if scen == 'historical':
                try:
                    zz = zz.sel(time=slice('1850-01-01','2014-12-31'))
                except:
                    # for 360 day calendars
                    zz = zz.sel(time=slice('1850-01-01','2014-12-30')) 
            else:
                try:
                    zz = zz.sel(time=slice('2015-01-01','2150-12-31'))
                except:
                    # for 360 day calendars
                    zz = zz.sel(time=slice('2015-01-01','2150-12-30')) 
                    
            if zz.time.dtype != '<M8[ns]':
                zz['time'] = zz.indexes['time'].to_datetimeindex()
    
            # calculate annual spatially-averaged temperatures

            # weight months by days in month
            month_length = zz.time.dt.days_in_month
            weights = (month_length.groupby("time.year") / 
                       month_length.groupby("time.year").sum())
            # Test that the sum of the weights for each season is 1.0
            np.testing.assert_allclose(weights.groupby("time.year").sum().
                                       values, np.ones(int(len(weights)/12))
                                       [0:len(weights.groupby("time.year").
                                              sum())])
            # Calculate the time-weighted average
            zz['tasann'] = (zz.tas * weights).groupby("time.year").sum(
                dim="time")

            # make sure that x and y are lon and lat
            try:
                zz = zz.rename({'latitude':'lat','longitude':'lon'})
            except:
                1
                
            # calculate the spatially weighted GMST
            zz = spatial_mean(zz)
            
            if scen=='historical':
                # get base period temperature
                base = zz.tasann.sel(year=slice('1850','1900')).mean(
                    'year').values
            
            # Add the GMST data to the output df
            yrs = zz.year.values
            ny = len(yrs)

            data = {'model': [mods[m]]*ny,
                    'member': [wrm.member]*ny,
                    'scenario': [scen]*ny, 
                    'year': yrs, 
                    'GMST': zz.tasann.values,
                    '1850-1900': [base]*ny}

            # Create mini dataframe for this model/scenario
            dfmini = pd.DataFrame(data, columns=['model','member','scenario',
                                                 'year','GMST','1850-1900'])
            
            # append this data for this model/scenario to the master GMST table
            outdf = pd.concat([outdf,dfmini])

/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This 

/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This 

/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/ipykernel_launcher.py:58: RuntimeWarning: Converting a CFTimeIndex with dates from a non-standard calendar, 'noleap', to a pandas.DatetimeIndex, which uses dates from the standard calendar.  This may lead to subtle errors in operations that depend on the length of time between dates.


/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/xarray/coding/times.py:526: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/home/abbylute/miniconda3/envs/warming_levels-env/lib/python3.7/site-packages/xarray/core/indexing.py:422: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(array[self.key], dtype=

In [11]:
outdf

,model,member,scenario,year,GMST,1850-1900
0,GFDL-CM4,r1i1p1f1,historical,1850,285.901631,285.915908201504
1,GFDL-CM4,r1i1p1f1,historical,1851,286.032717,285.915908201504
2,GFDL-CM4,r1i1p1f1,historical,1852,286.029380,285.915908201504
3,GFDL-CM4,r1i1p1f1,historical,1853,286.068782,285.915908201504
4,GFDL-CM4,r1i1p1f1,historical,1854,286.109054,285.915908201504
...,...,...,...,...,...,...
31,NorESM2-LM,r10i1p1f2,ssp245,2046,289.010323,NaN
32,NorESM2-LM,r10i1p1f2,ssp245,2047,288.895761,NaN
33,NorESM2-LM,r10i1p1f2,ssp245,2048,289.052433,NaN
34,NorESM2-LM,r10i1p1f2,ssp245,2049,289.021657,NaN


In [12]:
# There are some model/member combinations where the historical run wasn't 
# available, so 1850-1900=NaN. remove these
outdf.isna().sum() #35378 cases
outdf = outdf.loc[outdf['1850-1900'] > 0]
outdf

,model,member,scenario,year,GMST,1850-1900
0,GFDL-CM4,r1i1p1f1,historical,1850,285.901631,285.915908201504
1,GFDL-CM4,r1i1p1f1,historical,1851,286.032717,285.915908201504
2,GFDL-CM4,r1i1p1f1,historical,1852,286.029380,285.915908201504
3,GFDL-CM4,r1i1p1f1,historical,1853,286.068782,285.915908201504
4,GFDL-CM4,r1i1p1f1,historical,1854,286.109054,285.915908201504
...,...,...,...,...,...,...
81,ACCESS-CM2,r5i1p1f1,ssp585,2096,293.205425,287.0825326867011
82,ACCESS-CM2,r5i1p1f1,ssp585,2097,293.183995,287.0825326867011
83,ACCESS-CM2,r5i1p1f1,ssp585,2098,293.384004,287.0825326867011
84,ACCESS-CM2,r5i1p1f1,ssp585,2099,293.557349,287.0825326867011


In [14]:
# save the GMST table

outdf.to_csv(outfile, index=False)